# 4D-Lung

In [ ]:
import os
import cv2
import pydicom
import numpy as np

from PIL import Image
from pathlib import Path

In [ ]:
def get_file_list(path, suffix):
    return sorted([p for p in Path(path).rglob(suffix)])


def roi_idx(rt_dcm, roi='Tumor'):
    for i, seq in enumerate(rt_dcm.StructureSetROISequence):
        if roi in seq.ROIName:
            return i
    return -1
        
        
def proc_one_frame(ct_dcms, rt_dcm, outdir, patient_id, visit_date, frame_number, save_as_numpy=False):
    
    rt_dcm = pydicom.dcmread(rt_dcm)
    tumor_roi_idx = roi_idx(rt_dcm, roi='Tumor')
    if tumor_roi_idx == -1:
        return
    contour_dcm = rt_dcm.ROIContourSequence[tumor_roi_idx]
    
    # load uids for contour
    contour_uids = dict()
    for i in range(len(contour_dcm.ContourSequence)):
        uid = str(contour_dcm.ContourSequence[i].ContourImageSequence[0].ReferencedSOPInstanceUID)
        z_value = contour_dcm.ContourSequence[i].ContourData[2]
        contour_uids.update({uid: (i, z_value)})
    
    # match images to contours
    for ct_dcm in ct_dcms:
        ct_dcm = pydicom.dcmread(ct_dcm)
        ct_uid = str(ct_dcm.SOPInstanceUID)
        ct_z_value = ct_dcm.ImagePositionPatient[-1]
        
        if ct_uid not in contour_uids.keys():
            continue
        # assert contour_uids[ct_uid][1] == ct_z_value, f"{contour_uids[ct_uid][1]}, {ct_z_value}"

        # load pixel array from ct.dcm
        height = ct_dcm.Rows
        width = ct_dcm.Columns
        pixel_array = ct_dcm.PixelData
        
        rescale_slope = ct_dcm.RescaleSlope if 'RescaleSlope' in ct_dcm else 1
        rescale_intercept = ct_dcm.RescaleIntercept if 'RescaleIntercept' in ct_dcm else 0
        if ct_dcm.PixelRepresentation == 0:
            pixel_array = np.frombuffer(pixel_array, dtype=np.uint16).reshape((height, width))
        elif ct_dcm.PixelRepresentation == 1:
            pixel_array = np.frombuffer(pixel_array, dtype=np.int16).reshape((height, width))
        pixel_array = pixel_array * rescale_slope + rescale_intercept

        lower_limit = -910
        upper_limit = 590
        pixel_array = np.clip(pixel_array, lower_limit, upper_limit)

        pixel_array = (pixel_array - lower_limit) / (upper_limit - lower_limit)
        pixel_array = (pixel_array * 255).astype(np.uint8)
        img = Image.fromarray(pixel_array)
        
        # get segmentation from rt.dcm
        contour_num = contour_uids[ct_uid][0]
        contour = contour_dcm.ContourSequence[contour_num]
        spacing_x, spacing_y = ct_dcm.PixelSpacing
        origin_x, origin_y = ct_dcm.ImagePositionPatient[:2]
        
        points = []
        mask = np.zeros((height, width), dtype=np.uint8)
        for i in range(0, len(contour.ContourData), 3):
            x, y, _ = contour.ContourData[i:i+3]
            row = int((y - origin_y) / spacing_y)
            col = int((x - origin_x) / spacing_x)
            points.append([col, row])

        points = np.array(points, dtype=np.int32)
        cv2.fillPoly(mask, [points], color=255)
        label = Image.fromarray(mask)

        # save
        ct_slices = ct_dcm.InstanceNumber
        image_dir = os.path.join(outdir, 'image', patient_id, visit_date, f'Gated_{frame_number:02d}')
        os.makedirs(image_dir, exist_ok=True)
        label_dir = os.path.join(outdir, 'label', patient_id, visit_date, f'Gated_{frame_number:02d}')
        os.makedirs(label_dir, exist_ok=True)
        label.save(os.path.join(label_dir, f'1-{ct_slices:03d}.png'))
        img.save(os.path.join(image_dir, f'1-{ct_slices:03d}.png'))
        

In [ ]:
def data_proc_4d_lung(data_dir, outdir, debug=None):
    for patient_id in sorted(os.listdir(data_dir)):
        if ".DS" in patient_id:
            continue
        if "LICENSE" in patient_id:
            continue
        if debug:
            if patient_id != debug:
                continue
        print(patient_id)

        patient = os.path.join(data_dir, patient_id)
        for visit_date in sorted(os.listdir(patient)):
            if ".DS" in visit_date:
                continue
            if "LICENSE" in visit_date:
                continue
            print(visit_date)
            
            one_visit = os.path.join(patient, visit_date)
            video_label = sorted(os.listdir(one_visit))
            for i in range(0, len(video_label), 2):
                frame_label_pair = video_label[i: i+2]
                frame_path = frame_label_pair[1]
                label_path = frame_label_pair[0]
                if len(os.listdir(os.path.join(one_visit, label_path))) != 1:
                    frame_path = frame_label_pair[0]
                    label_path = frame_label_pair[1]
                    assert len(os.listdir(os.path.join(one_visit, label_path))) == 1
                print(label_path)
                ct_path = os.path.join(one_visit, frame_path)
                ct_path_list = get_file_list(ct_path, '*.dcm')
                rt_path = os.path.join(one_visit, label_path)
                rt_path = get_file_list(rt_path, '*.dcm')[0]
                
                proc_one_frame(ct_path_list, rt_path, outdir, patient_id, visit_date, i//2)

In [ ]:
# [HARD CODE]: 118_HM10395 is not correct, delete it.

data_proc_4d_lung('data/lung_org/4D-Lung-Multi-Visits', 'data/lung_pro/4d_lung_multi_visits')

# Nsclc-Radiomics

In [ ]:
import os
import pydicom
import numpy as np

from PIL import Image
from pathlib import Path

In [ ]:
def get_file_list(path, suffix):
    return sorted([p for p in Path(path).rglob(suffix)])


def dcm2img(ds, save_as_npy):
    pixel_array = ds.PixelData
    height = ds.Rows
    width = ds.Columns
    rescale_slope = ds.RescaleSlope if 'RescaleSlope' in ds else 1
    rescale_intercept = ds.RescaleIntercept if 'RescaleIntercept' in ds else 0
    
    if ds.PixelRepresentation == 0:
        pixel_array = np.frombuffer(pixel_array, dtype=np.uint16).reshape((height, width))
    elif ds.PixelRepresentation == 1:
        pixel_array = np.frombuffer(pixel_array, dtype=np.int16).reshape((height, width))
    pixel_array = pixel_array * rescale_slope + rescale_intercept

    lower_limit = -910
    upper_limit = 590
    pixel_array = np.clip(pixel_array, lower_limit, upper_limit)

    pixel_array = (pixel_array - lower_limit) / (upper_limit - lower_limit)
    if not save_as_npy:
        pixel_array = (pixel_array * 255).astype(np.uint8)
    else:
        pixel_array = pixel_array * 255
    return pixel_array


def dcm2seg(ds, save_as_npy):
    num_labels = len(ds.SegmentSequence)
    label_names = dict()
    for i in range(num_labels):
        label_names.update({ds.SegmentSequence[i].SegmentDescription: i})
    
    num_slices = ds.NumberOfFrames // num_labels
    original_shape = (num_labels, num_slices, 512, 512)
    
    pixel_array = ds.PixelData
    pixel_array = np.frombuffer(pixel_array, dtype=np.uint8)
    assert ds.HighBit == 0
    if ds.HighBit == 0:
        pixel_array = pixel_array[::-1]
    pixel_array = np.unpackbits(pixel_array) * 255
    pixel_array = pixel_array.reshape(original_shape)
    
    pixel_array = np.flip(pixel_array, axis=0)
    pixel_array = np.flip(pixel_array, axis=1)
    pixel_array = np.flip(pixel_array, axis=2)
    pixel_array = np.flip(pixel_array, axis=3)
    if not save_as_npy:
        pixel_array = pixel_array.astype(np.uint8)
    return pixel_array, ds.ReferencedSeriesSequence[0].ReferencedInstanceSequence, label_names

In [ ]:
def data_proc_nsclc_radiomics_tumor_class(data_dir, out_dir, save_as_npy=False):
    total_image = 0
    for lung_n in sorted(os.listdir(data_dir)):
        if ".DS" in lung_n:
            continue
        if "LICENSE" in lung_n:
            continue
            
        print("+", lung_n)
        image_label = os.path.join(data_dir, lung_n)
        study_id = os.listdir(image_label)
        study = os.path.join(image_label, study_id[0])
        assert len(os.listdir(study)) == 2
        study_seg = sorted(os.listdir(study))[1]
        study_img = sorted(os.listdir(study))[0]
        if 'Segmentation' not in study_seg:
            study_seg = sorted(os.listdir(study))[0]
            study_img = sorted(os.listdir(study))[1]
            assert 'Segmentation' in study_seg

        seg_path = os.path.join(study, study_seg)
        seg_path = get_file_list(seg_path, '*.dcm')[0]
        img_path = os.path.join(study, study_img)
        img_path_list = get_file_list(img_path, '*.dcm')

        dcm_seg = pydicom.dcmread(seg_path)
        seg_array, ref_seg_list, label_names = dcm2seg(dcm_seg, save_as_npy)

        num_images = 0
        for i, img_path in enumerate(img_path_list):
            dcm_img = pydicom.dcmread(img_path)
            ref_img = dcm_img.SOPInstanceUID
            slice_num = 0
            for j, ref_seg in enumerate(ref_seg_list):
                if ref_seg.ReferencedSOPInstanceUID == ref_img:
                    slice_num = j
                    break
            
            tumor_idx = label_names['GTV-1']
            if np.all(seg_array[tumor_idx, slice_num, :, :] == 0):
                continue
                
            tumor_array = seg_array[tumor_idx, slice_num, :, :]
            segment_dir = os.path.join(out_dir, 'label', lung_n)
            os.makedirs(segment_dir, exist_ok=True)
            if not save_as_npy:
                img = Image.fromarray(tumor_array)
                img.save(os.path.join(segment_dir, f'1-{i:03d}.png'))
            else:
                np.save(os.path.join(segment_dir, f'1-{i:03d}.npy'), tumor_array)
            
            img_array = dcm2img(dcm_img, save_as_npy)
            image_dir = os.path.join(out_dir, 'image', lung_n)
            os.makedirs(image_dir, exist_ok=True)
            if not save_as_npy:
                img = Image.fromarray(img_array)
                img.save(os.path.join(image_dir, f'1-{i:03d}.png'))
            else:
                np.save(os.path.join(image_dir, f'1-{i:03d}.npy'), img_array)
            
            num_images+=1
        print(num_images)
        total_image += num_images
    print(total_image)

In [ ]:
data_proc_nsclc_radiomics_tumor_class('data/lung_org/NSCLC-Radiomics', 'data/lung_pro/nsclc_radiomics')